In [36]:
import pandas as pd
import numpy as np
import json
import os
from tabulate import tabulate

In [31]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQD\Destructive SWAP\Files"
cutoffs = [2,4,8,16,32]
potentials = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [40]:
converged_only = False
all_data = []

for potential in potentials:
    for cutoff in cutoffs:

        median = []
        mean = []
        min = []
        e0 = []
        e1 = []
        e2 = []
        ratios = []

        data_path = folder_path + "\\{}\\{}_{}.json".format(potential,potential, cutoff)
        
        with open(data_path, 'r') as file:
            data = json.load(file)

        num_VQD = data['num_VQD']

        converged = [all(x) for x in data['success']]
        converged_indices = [i for i, success in enumerate(converged) if success]

        if converged_only:
            results = [data['results'][i] for i in converged_indices]
        else:
            results = data['results']

        success_i = [i for i, success in enumerate(data['success']) if all(success)]
        successful_results = [data['results'][i] for i in success_i]
        energies =np.sort(successful_results)

        for e in energies:
            
            e0.append(e[0])
            e1.append(e[1])
            e2.append(e[2])
            r = abs((e[2] - e[1]) / (e[2] - e[0]))
            ratios.append(r)

        e0_exact = data['exact_eigenvalues'][0]
        e1_exact = data['exact_eigenvalues'][1]
        e2_exact = data['exact_eigenvalues'][2]
        exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

        row = {
        "potential": potential,
        "cutoff": cutoff,
        "e0_exact": e0_exact,
        "e1_exact": e1_exact,
        "e2_exact": e2_exact,
        "exact_ratio": exact_ratio,
        "e0": np.median(e0),
        "e1": np.median(e1),
        "e2": np.median(e2),
        "median_ratio": np.median(ratios),
        "mean_ratio": np.mean(ratios),
        "min_ratio": np.min(ratios)
        }

        all_data.append(row)

In [41]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,e0_exact,e1_exact,e2_exact,exact_ratio,e0,e1,e2,median_ratio,mean_ratio,min_ratio
0,AHO,2,-0.437500,-0.437500,2.062500,1.000000e+00,-0.466178,-0.437500,1.473307,0.985373,0.985089,0.976634
1,AHO,4,-0.164785,0.673310,1.667943,5.427061e-01,-0.164785,0.648749,1.627111,0.544890,0.543794,0.515699
2,AHO,8,0.032010,1.680156,1.833526,8.513391e-02,0.032010,1.655412,1.807187,0.086568,0.090137,0.027300
3,AHO,16,-0.001167,1.677494,1.686381,5.266251e-03,-0.001167,1.657257,1.674103,0.008390,0.042952,0.000063
4,AHO,32,0.000006,1.686499,1.686559,3.585457e-05,0.000006,1.658554,1.676974,0.010605,0.037657,0.000251
5,DW,2,0.357233,0.771447,1.478553,6.306019e-01,0.357233,0.747672,1.448140,0.642145,0.642394,0.620660
6,DW,4,0.906560,0.950634,1.695666,9.441474e-01,0.906560,0.950354,1.674712,0.939651,0.943241,0.897188
7,DW,8,0.884580,0.887725,2.693873,9.982617e-01,0.873623,0.889486,2.784411,0.990985,0.990412,0.948532
8,DW,16,0.891599,0.891641,2.734122,9.999774e-01,0.881755,0.893399,2.780625,0.992269,0.991184,0.963387
9,DW,32,0.891632,0.891632,2.734011,1.000000e+00,0.879497,0.891632,2.791916,0.992342,0.992059,0.976970


In [39]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex", floatfmt=".3f")
print(latex_table)

\begin{tabular}{lrrrrrrrrrrr}
\hline
 potential   &   cutoff &   e0\_exact &   e1\_exact &   e2\_exact &   exact\_ratio &     e0 &     e1 &    e2 &   median\_ratio &   mean\_ratio &   min\_ratio \\
\hline
 AHO         &        2 &     -0.437 &     -0.437 &      2.062 &         1.000 & -0.466 & -0.437 & 1.473 &          0.985 &        0.985 &       0.977 \\
 AHO         &        4 &     -0.165 &      0.673 &      1.668 &         0.543 & -0.165 &  0.649 & 1.627 &          0.545 &        0.544 &       0.516 \\
 AHO         &        8 &      0.032 &      1.680 &      1.834 &         0.085 &  0.032 &  1.655 & 1.807 &          0.087 &        0.090 &       0.027 \\
 AHO         &       16 &     -0.001 &      1.677 &      1.686 &         0.005 & -0.001 &  1.657 & 1.674 &          0.008 &        0.043 &       0.000 \\
 AHO         &       32 &      0.000 &      1.686 &      1.687 &         0.000 &  0.000 &  1.659 & 1.677 &          0.011 &        0.038 &       0.000 \\
 DW          &        2 &